Please note that I'm mostly a C++ dev; I'm not the best with Python and data
analysis, so this code is a bit more sloppy that I'd like it to be.  My apologies.

Also, due to a bug, the scene `book1::glass_blue_metal_spheres` was doubly tested.
I don't think it makes sense to prune one of the runs from the testing data.  When
analyzing it, nothing looks to be out of the ordinary or swaying the results in one
direction over the other.  If anything it actually gives more more data to inspect
and average later on.  It's the rare time where a bug actually became a feature!
(I will be fixing it and updating the test cases later on in the nearish future)

In [1]:
from openpyxl import load_workbook
from dataclasses import dataclass
import pandas as pd
import numpy as np
import matplotlib as plt
from pprint import pprint

num_test_cases = 1170   # TODO, it would be nice to somehow grab this value from the spreadsheet and not hard code it...
sheet_file_name ='final_keyword_test_suite_results.xlsx'
excel_file = load_workbook(sheet_file_name)

In [2]:
from dataclasses import dataclass

@dataclass
class TestCase:
    id_num: int
    scene_id: str

    # runtimes, measured in nanoseconds
    without_final_runtime_ns: int
    with_final_runtime_ns: int

    def was_final_faster(self) -> bool:
        '''Checks if use of final was faster for this'''
        return (self.with_final_runtime_ns < self.without_final_runtime_ns)

    def final_slower_by_ns(self) -> int:
        '''How much slower using `final` was. Could return a negative, meaning final was faster'''
        return self.with_final_runtime_ns - self.without_final_runtime_ns

    def final_faster_by_ns(self) -> int:
        '''How much final was faser by, may return a negative meaning it was slower'''
        return -self.final_slower_by_ns()

    def percent_final_slower_by(self) -> float:
        '''Percent-wise how much slower final was'''
        return (self.with_final_runtime_ns - self.without_final_runtime_ns) / self.without_final_runtime_ns

    def percent_final_faster_by(self) -> float:
        '''Percent-wise how much faster final was'''
        return (self.without_final_runtime_ns - self.with_final_runtime_ns) / self.without_final_runtime_ns

In [3]:
def _ns_to_time_str(nanoseconds: int) -> str:
    total_seconds = nanoseconds // 1000000000
    hours = total_seconds // (60 * 60)
    s = total_seconds % (60 * 60)
    minutes = s // 60
    seconds = s % 60
    
    return f'{hours}h {minutes}m {seconds}s'
    

@dataclass
class TestSuite:
    cpu: str
    os: str
    compiler: str

    # Measured in nanoseconds
    without_final_total_runtime_ns: int
    with_final_total_runtime_ns: int

    test_cases: list[TestCase]

    def without_final_total_runtime_str(self) -> str:
        return _ns_to_time_str(self.without_final_total_runtime_ns)
        
    def with_final_total_runtime_str(self) -> str:
        return _ns_to_time_str(self.with_final_total_runtime_ns)

    def pandas_column_header(self) -> str:
        '''For displaying the configuration in a header in a table'''
        return '<br>'.join([self.compiler, self.cpu, self.os])

    def overall_was_final_more_performant(self) -> bool:
        return (self.with_final_total_runtime_ns < self.without_final_total_runtime_ns)

    def overall_how_much_faster_was_final(self) -> float:
        # Assumes that final was always faster
        diff = self.without_final_total_runtime_ns - self.with_final_total_runtime_ns
        return (diff / self.without_final_total_runtime_ns)

    def _find_cases_where(self, comparison_func) -> list[TestCase]:
        '''Internal function to compare the final vs. non-used-final test case,
        and collect them into a list.  comparision_func takes one arguement (a test
        case) and should return a bool'''
        return list(filter(lambda x: comparison_func(x), self.test_cases))
        
    def find_cases_where_final_was_slower(self) -> list[TestCase]:
        '''Retuns a list of test cases where the usage of `final` was slower'''
        def _final_slower(test_case):
            return (test_case.with_final_runtime_ns >= test_case.without_final_runtime_ns)

        return self._find_cases_where(_final_slower)

    def find_cases_where_final_was_faster(self) -> list[TestCase]:
        '''Returns a list of test cases where the usage of `final` was faster'''
        def _final_faster(test_case):
            return (test_case.with_final_runtime_ns < test_case.without_final_runtime_ns)

        return self._find_cases_where(_final_faster)        

    def find_cases_where_final_was_slower_by_at_least(self, by_percent: float) -> list[TestCase]:
        '''Find where the test case was slower with `final` turned on;  By a certain percentage'''
        all_where_slower = self.find_cases_where_final_was_slower()
        return list(filter(lambda x: (x.percent_final_slower_by() >= by_percent), all_where_slower))
        
    def find_cases_where_final_was_faster_by_at_least(self, by_percent: float) -> list[TestCase]:
        '''Find where the test case was faster with `final` turned on;  By a certain percentage'''
        all_where_faster = self.find_cases_where_final_was_faster()
        return list(filter(lambda x: (x.percent_final_faster_by() >= by_percent), all_where_faster))

    def find_cases_that_took_at_least(self, how_many_seconds: float) -> list[TestCase]:
        '''Tries to find the test cases where with/without final it took a few seconds'''
        def _at_least_n_seconds(test_case):
            final_runtime_s = test_case.with_final_runtime_ns / 1000000000
            without_final_runtime_s = test_case.without_final_runtime_ns / 1000000000
            return ((final_runtime_s > how_many_seconds) or (without_final_runtime_s > how_many_seconds))

        return self._find_cases_where(_at_least_n_seconds)

In [4]:
results = []
num_configurations = 0

def pairwise(iterable):
    a = iter(iterable)
    return zip(a, a)

# Parse the data into something we can use/analyze
for (without_final, with_final) in pairwise(excel_file.worksheets):
    num_configurations += 1
    
    # Extract out the summary data
    without_final_summary_data = without_final['A1:B5']
    with_final_summary_data = with_final['A1:B5']

    # Grab the configuration (and results of each individual test)
    data_start_offset = 8
    range_to_grab = f'A{data_start_offset}:I{data_start_offset + num_test_cases - 1}'
    without_final_test_data = without_final[range_to_grab]
    with_final_test_data = with_final[range_to_grab]
    
    # Get all the data from each test case
    test_cases = []
    for r in range(0, num_test_cases):
        test_cases.append(TestCase(
            int(without_final_test_data[r][0].value),      # id no.
            without_final_test_data[r][1].value,           # scene name

            # Render time
            int(without_final_test_data[r][8].value),
            int(with_final_test_data[r][8].value)
        ))

    # Put them into the suite results
    results.append(TestSuite(
        without_final_summary_data[0][1].value,
        without_final_summary_data[1][1].value,
        without_final_summary_data[2][1].value,
        
        int(without_final_summary_data[4][1].value),
        int(with_final_summary_data[4][1].value),
        test_cases
    ))

In [5]:
# Make a chart showing the overall performance
configs = [x.pandas_column_header() for x in results]
data = [
    [('Yes' if x.overall_was_final_more_performant() else 'No') for x in results],
    [('{:.2%}'.format(x.overall_how_much_faster_was_final())) for x in results],
    [x.without_final_total_runtime_str() for x in results],
    [x.with_final_total_runtime_str() for x in results],
]
row_labels = [
    'Was <code>final</code> faster?',
    'By how much',
    'Total run time without <code>final</final>',
    'Total run time with <code>final</final>',
]
overall_table = pd.DataFrame(columns=configs, data=data)
overall_table.style\
  .set_caption(f'<h1>Overall Performance</h1>')\
  .relabel_index(row_labels, axis=0)

,GCC 13.2.0AMD Ryzen 9 6900HXUbuntu 23.10,Clang 17.0.2AMD Ryzen 9 6900HXUbuntu 23.10,MSVC 17AMD Ryzen 9 6900HXWindows 11 Home (22631.3085),GCC 13.2.0 (w64devkit)AMD Ryzen 9 6900HXWindows 11 Home (22631.3085),Clang 15M1macOS 14.3 (23D56),GCC 13.2.0 (homebrew)M1macOS 14.3 (23D56),GCC 12.3.0i7-10750HUbuntu 22.04.3
Was final faster?,Yes,No,No,Yes,Yes,Yes,Yes
By how much,3.62%,-9.91%,-3.66%,3.57%,0.05%,1.94%,0.44%
Total run time without final,8h 8m 11s,12h 19m 49s,11h 48m 53s,10h 22m 54s,5h 21m 22s,5h 38m 50s,8h 46m 59s
Total run time with final,7h 50m 30s,13h 33m 10s,12h 14m 48s,10h 0m 40s,5h 21m 12s,5h 32m 17s,8h 44m 39s


In [6]:
def make_table_showing_performance_percentages(slower_or_faster: str, lower: int = 1, upper: int = 5):
    '''This function will generate a table that will show for each configuration, how
    many tests (percentage) wise are faster or slower with `final` on or off.  It's
    a little confusing TBH.  First arugument to this function should be `slower` or `faster`;
    the second and third arguments are the range to show (inclusive).'''

    # Decide on the colour, and function to use to grab tests
    if slower_or_faster == 'slower':
        cmap = 'YlOrRd'
        f = TestSuite.find_cases_where_final_was_slower_by_at_least
    elif slower_or_faster == 'faster':
        cmap = 'Greens'
        f = TestSuite.find_cases_where_final_was_faster_by_at_least

    # Percentages to test
    perf_percentages = [(x / 100) for x in range(lower, (upper + 1))]
    
    row_labels = []
    data = []
    for pp in perf_percentages:
        row_labels.append(f'≥ {pp:.0%} {slower_or_faster}')
        row_data = []

        # Figure out the percentage of test cases, at or above the performance level
        for r in results:
            cases = f(r, pp)
            ratio = len(cases) / num_test_cases
            row_data.append(ratio)
    
        data.append(row_data)
    
    def _make_pretty(styler):
        styler.set_caption(f'<h2>Percent of test cases that are {slower_or_faster} with <code>final</code> turned on</h2><h4>(out of {num_test_cases} test cases)</h4><br><br>')
        styler.relabel_index(row_labels, axis=0)
        styler.format('{:.1%}')
        styler.background_gradient(axis=None, cmap=cmap, vmin=0, vmax=1)
        return styler
    
    # Display the data
    table = pd.DataFrame(columns=configs, data=data)
    return (table, _make_pretty)

In [7]:
# Show the faster cases
(table, make_pretty) = make_table_showing_performance_percentages('faster', 1, 10)
table.style.pipe(make_pretty)

,GCC 13.2.0AMD Ryzen 9 6900HXUbuntu 23.10,Clang 17.0.2AMD Ryzen 9 6900HXUbuntu 23.10,MSVC 17AMD Ryzen 9 6900HXWindows 11 Home (22631.3085),GCC 13.2.0 (w64devkit)AMD Ryzen 9 6900HXWindows 11 Home (22631.3085),Clang 15M1macOS 14.3 (23D56),GCC 13.2.0 (homebrew)M1macOS 14.3 (23D56),GCC 12.3.0i7-10750HUbuntu 22.04.3
≥ 1% faster,66.3%,3.8%,32.4%,35.4%,14.5%,22.7%,29.7%
≥ 2% faster,49.0%,3.5%,25.5%,25.2%,5.4%,10.6%,23.0%
≥ 3% faster,36.0%,2.9%,19.9%,18.2%,2.6%,6.5%,15.7%
≥ 4% faster,22.1%,2.6%,16.4%,14.0%,1.8%,5.5%,13.6%
≥ 5% faster,15.8%,2.5%,13.3%,10.6%,1.6%,5.4%,12.2%
≥ 6% faster,10.6%,2.3%,11.8%,7.5%,1.2%,4.5%,11.0%
≥ 7% faster,7.3%,2.0%,10.4%,5.0%,0.8%,3.4%,10.3%
≥ 8% faster,5.6%,1.9%,9.5%,3.8%,0.7%,1.5%,9.6%
≥ 9% faster,5.0%,1.7%,8.3%,3.2%,0.7%,0.6%,9.0%
≥ 10% faster,4.4%,1.5%,7.2%,3.0%,0.6%,0.5%,8.6%


In [8]:
# Show the slower cases
(table, make_pretty) = make_table_showing_performance_percentages('slower', 1, 10)
table.style.pipe(make_pretty)

,GCC 13.2.0AMD Ryzen 9 6900HXUbuntu 23.10,Clang 17.0.2AMD Ryzen 9 6900HXUbuntu 23.10,MSVC 17AMD Ryzen 9 6900HXWindows 11 Home (22631.3085),GCC 13.2.0 (w64devkit)AMD Ryzen 9 6900HXWindows 11 Home (22631.3085),Clang 15M1macOS 14.3 (23D56),GCC 13.2.0 (homebrew)M1macOS 14.3 (23D56),GCC 12.3.0i7-10750HUbuntu 22.04.3
≥ 1% slower,11.3%,94.3%,44.4%,29.9%,12.3%,11.5%,25.4%
≥ 2% slower,6.5%,92.9%,39.1%,18.8%,5.1%,5.8%,19.6%
≥ 3% slower,3.2%,91.2%,35.0%,12.7%,2.5%,3.6%,15.1%
≥ 4% slower,1.5%,90.2%,30.4%,9.5%,1.2%,1.9%,12.6%
≥ 5% slower,0.9%,89.1%,26.2%,7.1%,0.7%,0.8%,10.6%
≥ 6% slower,0.5%,87.5%,22.6%,6.3%,0.5%,0.6%,9.8%
≥ 7% slower,0.3%,85.1%,21.2%,5.4%,0.3%,0.5%,9.5%
≥ 8% slower,0.3%,82.7%,20.4%,4.9%,0.3%,0.3%,9.1%
≥ 9% slower,0.3%,80.3%,19.4%,4.5%,0.1%,0.3%,8.6%
≥ 10% slower,0.1%,78.2%,18.5%,4.1%,0.1%,0.3%,8.2%


In [9]:
# Collapse each iterate of each scene to show averages to see if it was more performant or not
from statistics import mean, median, stdev, variance
from collections import OrderedDict

# Not a fan of doing this, but it extracts the scene Id's for us to display in the table
_scene_ids = []

def compute_stats_per_config(config_index: int):
    global _scene_ids
    
    suite = results[config_index]
    all_scenes = OrderedDict()
    
    # Organize each scene to see how much percent faster it is
    for tc in suite.test_cases:
        sid = tc.scene_id
        p_faster = tc.percent_final_faster_by()
        
        if sid in all_scenes:
            all_scenes[sid].append(p_faster)  # Add value
        else:
            all_scenes[sid] = [p_faster]      # New list
            
    data = []
    scene_ids = []
    for (scene_id, measurements) in all_scenes.items():
        scene_ids.append(scene_id)
        # Do math
        data.append({
            f'Average_{config_index}': mean(measurements),
            f'Median_{config_index}': median(measurements),
#            f'Variance_{config_index}': variance(measurements),  # This column is left out of the final table, as it's typically low.  there are some cases where it is high
        })

    _scene_ids = scene_ids
        
    return pd.DataFrame(data=data)

# Take the averages of the performance increase/hit and put them into a super table
perf_avgs = [compute_stats_per_config(x) for x in range(0, num_configurations)]

# There is a +1 to account for a scene being doubly tested.  See my note at the
# top;  It does not effect the results in a poor manner
num_test_cases_per_scene = num_test_cases // (len(_scene_ids) + 1)

print('I was having some issue createing a Multi-Index table (from arrays)')
print('And having the table be both styled and formatted nicely.  So instead')
print('each column has a configuration number appened the end of its name.')
print('Here is hwat each number means:')
for (i, config) in enumerate(configs):
    c = config.replace('<br>', ' ')
    print(f'  {i}: {c}')

super_table = pd.concat(perf_avgs, axis=1)
super_table.style\
  .format('{:.1%}')\
  .background_gradient(axis=None, cmap='RdBu', vmin=-0.1, vmax=0.1)\
  .relabel_index(_scene_ids, axis=0)\
  .set_caption(f'<h2>How much faster/slower was each scene on average if <code>final</code> was used</h2><h4>(Each scene being tested at least {num_test_cases_per_scene} times)</h4><br><br>')

I was having some issue createing a Multi-Index table (from arrays)
And having the table be both styled and formatted nicely.  So instead
each column has a configuration number appened the end of its name.
Here is hwat each number means:
  0: GCC 13.2.0 AMD Ryzen 9 6900HX Ubuntu 23.10
  1: Clang 17.0.2 AMD Ryzen 9 6900HX Ubuntu 23.10
  2: MSVC 17 AMD Ryzen 9 6900HX Windows 11 Home (22631.3085)
  3: GCC 13.2.0 (w64devkit) AMD Ryzen 9 6900HX Windows 11 Home (22631.3085)
  4: Clang 15 M1 macOS 14.3 (23D56)
  5: GCC 13.2.0 (homebrew) M1 macOS 14.3 (23D56)
  6: GCC 12.3.0 i7-10750H Ubuntu 22.04.3


,Average_0,Median_0,Average_1,Median_1,Average_2,Median_2,Average_3,Median_3,Average_4,Median_4,Average_5,Median_5,Average_6,Median_6
book1::surface_normal_sphere,1.2%,0.9%,-26.9%,-26.2%,-0.4%,-0.2%,0.2%,0.3%,0.2%,-0.1%,-0.7%,-0.3%,-3.4%,-2.4%
book1::grey_sphere,1.9%,1.1%,-25.6%,-24.8%,-0.1%,-0.4%,-1.1%,-0.8%,-0.4%,-0.1%,-0.9%,-0.3%,-0.5%,-0.1%
book1::shiny_metal_sphere,0.6%,0.8%,-17.2%,-18.5%,-0.0%,0.4%,-0.0%,-0.0%,-0.7%,-0.1%,-0.4%,-0.0%,-1.0%,0.0%
book1::fuzzy_metal_sphere,2.6%,1.7%,-17.5%,-17.8%,0.4%,-0.1%,-0.6%,-0.3%,-0.3%,-0.0%,-0.5%,-0.4%,0.3%,0.0%
book1::two_glass_one_metal_spheres,1.1%,0.8%,-21.1%,-20.8%,0.6%,0.3%,0.4%,0.0%,2.3%,0.1%,0.0%,0.2%,-3.1%,-0.3%
book1::glass_blue_metal_spheres,2.2%,2.1%,-18.9%,-19.8%,1.0%,0.5%,-0.5%,-0.0%,-0.1%,0.0%,-0.0%,-0.1%,-0.3%,-0.0%
book1::red_blue_spheres,0.9%,0.3%,-29.6%,-30.7%,1.0%,0.9%,-0.4%,-0.6%,1.4%,0.7%,-1.3%,-0.5%,-0.4%,-0.0%
book1::glass_blue_metal_spheres::far_view,1.6%,1.3%,-27.3%,-28.2%,0.5%,0.4%,1.6%,1.4%,0.4%,0.4%,-0.5%,-0.3%,0.8%,0.9%
book1::glass_blue_metal_spheres::close_view,4.2%,2.9%,-15.3%,-15.3%,1.5%,1.2%,0.1%,0.4%,0.1%,0.1%,-0.0%,-0.2%,1.9%,0.0%
book1::glass_blue_metal_spheres::close_fuzzy_view,2.0%,1.8%,-18.3%,-18.1%,3.5%,1.5%,0.1%,0.5%,0.3%,0.3%,-0.7%,-0.4%,1.0%,0.0%
